In [ ]:
# HIDE
import  matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from progressbar import progressbar
import sys

In [ ]:
x = np.random.uniform(size=(10000, 10))
y = x
# y = np.random.normal(x, 0.1, size=(100000, 1))

In [ ]:
class MyDataSet(TensorDataset):
    def __init__(self, x, y):
        self.xt = torch.from_numpy(x).float()
        self.yt = torch.from_numpy(y).float()
        super(MyDataSet, self).__init__(self.xt, self.yt)
        
my_ds = MyDataSet(x, y)
dl = DataLoader(my_ds, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
my_ds.xt 

In [ ]:
for n_hidden in [1, 4, 6, 8, 9]:
    
    print(f'\nn_hidden = {n_hidden}')
    reg = nn.Sequential(nn.Linear(x.shape[1], n_hidden),
                        nn.Linear(n_hidden, x.shape[1]))
    loss = nn.MSELoss()
    optimizer = optim.Adam(reg.parameters())    
    
    for epoch in range(10):
        for x, y in dl:
            reg.zero_grad()
            py = reg(x)
            l = loss(py, y)
            sys.stdout.write(f'\r Epoch {epoch}: {l}')
            sys.stdout.flush()
            l.backward()
            optimizer.step()

    pd.Series(torch.mean((reg(my_ds.xt) - my_ds.yt)**2, dim=1).detach().numpy()).hist(bins=100, alpha=0.3, label=f'{n_hidden}')
    
plt.legend()